In [2]:
import pandas as pd
# import datetime
import pandas as pd
import statistics
import numpy as np
from datetime import datetime
import re
import pandas as pd
import numpy as np

In [3]:
case_column = "case concept:name"
registration_time_column = "case REG_DATE"
event_column = "event concept:name"
timestamp_column = "event time:timestamp"
position_column = "Position"
baseline_next_event_column = "Baseline Prediction for Next Activity"
pos_event_next_event_column = "Position&Event Prediction for Next Activity"
baseline_next_timestamp_column = "Baseline Prediction for Next Timestamp"
pos_event_next_timestamp_column = "Position&Event Prediction for Next Timestamp"
time_since_registration_column = "Time Since Registration"
day_of_week_column = "day_of_week"
month_of_year_column = "month_of_year"
offer_sent = 'O_SENT' # the name a of a sent offer state within event_column
timeformat_registration = "%Y-%m-%dT%H:%M:%S" # new time format
timeformat_timestamp = "%d-%m-%Y %H:%M:%S.%f"
predicted_event_shift_column = 'Predicted event from baseline'
predicted_time_shift_column = 'Predicted time from baseline'

In [4]:
df_train = pd.read_csv('DATA/BPI Challenge 2017-training.csv')
df_test = pd.read_csv('DATA/BPI Challenge 2017-test.csv')

In [5]:
df_train[timestamp_column] = [datetime.strptime(date, timeformat_timestamp) for date in df_train[timestamp_column]]
df_train = df_train.sort_values(by=[case_column, timestamp_column])  

df_test[timestamp_column] = [datetime.strptime(date, timeformat_timestamp) for date in df_test[timestamp_column]]
df_test = df_test.sort_values(by=[case_column, timestamp_column]) 

In [6]:
df_train[day_of_week_column] = df_train[timestamp_column].dt.day_name()
df_train['hour']  = [i.hour for i in df_train[timestamp_column]]

df_test[day_of_week_column] = df_test[timestamp_column].dt.day_name()
df_test['hour']  = [i.hour for i in df_test[timestamp_column]]

In [7]:
df_train[position_column] = df_train.groupby([case_column]).cumcount()+1

df_test[position_column] = df_test.groupby([case_column]).cumcount()+1

In [8]:
df_test

,eventID,case LoanGoal,case ApplicationType,case concept:name,case RequestedAmount,event Action,event org:resource,event concept:name,event EventOrigin,event EventID,event lifecycle:transition,event time:timestamp,day_of_week,hour,Position
104660,123214021787648,Car,New credit,Application_1000386745,5000.0,Created,User_1,A_Create Application,Application,Application_1000386745,complete,2016-11-25 15:31:09.852,Friday,15,1
104661,123214021787649,Car,New credit,Application_1000386745,5000.0,statechange,User_1,A_Submitted,Application,ApplState_508603983,complete,2016-11-25 15:31:11.088,Friday,15,2
104662,123214021787650,Car,New credit,Application_1000386745,5000.0,Created,User_1,W_Handle leads,Workflow,Workitem_1662375286,schedule,2016-11-25 15:31:11.309,Friday,15,3
104677,123214021787651,Car,New credit,Application_1000386745,5000.0,Deleted,User_1,W_Handle leads,Workflow,Workitem_1449448208,withdraw,2016-11-25 15:32:40.415,Friday,15,4
104678,123214021787652,Car,New credit,Application_1000386745,5000.0,Created,User_1,W_Complete application,Workflow,Workitem_585834251,schedule,2016-11-25 15:32:40.421,Friday,15,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182493,130438156779538,Existing loan takeover,New credit,Application_999632431,12000.0,Deleted,User_1,W_Call after offers,Workflow,Workitem_1894495253,ate_abort,2016-12-20 07:32:24.316,Tuesday,7,19
182494,130438156779539,Existing loan takeover,New credit,Application_999632431,12000.0,Created,User_1,W_Call after offers,Workflow,Workitem_1671270331,schedule,2016-12-20 07:32:24.326,Tuesday,7,20
235439,130438156779540,Existing loan takeover,New credit,Application_999632431,12000.0,statechange,User_1,A_Cancelled,Application,ApplState_318084843,complete,2017-01-16 08:00:30.429,Monday,8,21
235440,130438156779541,Existing loan takeover,New credit,Application_999632431,12000.0,statechange,User_1,O_Cancelled,Offer,OfferState_560115325,complete,2017-01-16 08:00:30.449,Monday,8,22


In [9]:
df_train['next_event_time'] = df_train[timestamp_column].shift(-1)

df_test['next_event_time'] = df_test[timestamp_column].shift(-1)

In [10]:
df_train['time until next'] = df_train['next_event_time'] - df_train[timestamp_column]
df_train['minutes until next'] = [i.total_seconds()/60 for i in df_train['time until next']]

df_test['time until next'] = df_test['next_event_time'] - df_test[timestamp_column]
df_test['minutes until next'] = [i.total_seconds()/60 for i in df_test['time until next']]

In [11]:
df_train = df_train.dropna()

df_test = df_test.dropna()

In [12]:
# lstm_model=Sequential()
# lstm_model.add(LSTM(units=50,activation='relu',input_shape=(np.shape(X)[1],1)))
# lstm_model.add(Dense(1))
# lstm_model.compile(optimizer='adam', loss='mse')
# lstm_model.fit(X, y, epochs=1,batch_size=1,verbose=2)

In [13]:
X = df_train[['case LoanGoal', 'case ApplicationType', 'case RequestedAmount', 'event concept:name', 'event EventOrigin', 'event lifecycle:transition', 'hour',
       'day_of_week', 'Position']]
y = df_train['minutes until next']
# y = np.asarray(y).astype('float64')

In [14]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
 
label = le.fit_transform(X['case LoanGoal'])
X['case LoanGoal'] = label

label = le.fit_transform(X['case ApplicationType'])
X['case ApplicationType'] = label

label = le.fit_transform(X['event concept:name'])
X['event concept:name'] =  label

label = le.fit_transform(X['day_of_week'])
X['day_of_week'] =  label

label = le.fit_transform(X['event EventOrigin'])
X['event EventOrigin'] =  label

label = le.fit_transform(X['event lifecycle:transition'])
X['event lifecycle:transition'] =  label

<ipython-input-14-6c35ce177fc1>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['case LoanGoal'] = label
<ipython-input-14-6c35ce177fc1>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['case ApplicationType'] = label
<ipython-input-14-6c35ce177fc1>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

In [15]:
X_test = df_test[['case LoanGoal', 'case ApplicationType', 'case RequestedAmount', 'event concept:name', 'event EventOrigin', 'event lifecycle:transition', 'hour',
       'day_of_week', 'Position']]
y_test = df_test['minutes until next']
# y = np.asarray(y).astype('float64')

le = LabelEncoder()
 
label = le.fit_transform(X_test['case LoanGoal'])
X_test['case LoanGoal'] = label

label = le.fit_transform(X_test['case ApplicationType'])
X_test['case ApplicationType'] =  label

label = le.fit_transform(X_test['event concept:name'])
X_test['event concept:name'] = label

label = le.fit_transform(X_test['day_of_week'])
X_test['day_of_week'] =  label

label = le.fit_transform(X_test['event EventOrigin'])
X_test['event EventOrigin'] =  label

label = le.fit_transform(X_test['event lifecycle:transition'])
X_test['event lifecycle:transition'] =  label

<ipython-input-15-89754eeab5b3>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['case LoanGoal'] = label
<ipython-input-15-89754eeab5b3>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['case ApplicationType'] =  label
<ipython-input-15-89754eeab5b3>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

In [16]:
X

,case LoanGoal,case ApplicationType,case RequestedAmount,event concept:name,event EventOrigin,event lifecycle:transition,hour,day_of_week,Position
651461,10,1,5000.0,4,0,1,17,6,1
651462,10,1,5000.0,8,0,1,17,6,2
651463,10,1,5000.0,22,2,3,17,6,3
651465,10,1,5000.0,22,2,6,17,6,4
651466,10,1,5000.0,21,2,3,17,6,5
...,...,...,...,...,...,...,...,...,...
932615,3,1,30000.0,20,2,5,14,6,30
934575,3,1,30000.0,20,2,2,10,4,31
934589,3,1,30000.0,20,2,5,10,4,32
940763,3,1,30000.0,10,1,1,8,1,33


In [17]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
regressor.fit(X, y)

RandomForestRegressor(random_state=0)

In [18]:
predictions = regressor.predict(X_test)
errors = abs(predictions - y_test)
print('Mean Absolute Error:', round(np.mean(errors), 2))

Mean Absolute Error: 5921.3


In [20]:
5921.3/60

98.68833333333333